In [ ]:
"""
Pytorch是Facebook研究团队所开发的一款深度学习框架，具备自主性强，易操作，模型稳定等优点，类似的框架还有TensorFlow

下面是一些关于Pytorch进行深度学习的简单介绍和教程：
1.https://www.zhihu.com/collection/710418581?page=2
2.https://blog.csdn.net/sinat_39448069/article/details/120866541?spm=1001.2014.3001.5506
3.https://blog.csdn.net/weixin_44216612/article/details/124203730?spm=1001.2014.3001.5506

基于Pytorch进行深度学习主要分为以下几步：
1. 封装并加载数据集
2. 构建神经网络模型（网络框架、损失函数，损失优化策略）
3. 模型的训练（训练函数，训练过程，有时候还需要调超参数）
4. 保存模型
5. 模型预测

1.Dataset
    1.1.batch_size
    1.2.transform
2.DataLoader
3.Model
    3.1.network
    3.2.loss
    3.3.optimizer
        3.3.1.Learning rate
        3.3.2.Early stopping
4.分布式模型改造
5.超参数优化
6.checkpoint
7.train 函数
    7.1.Accuracy, recall 计算
    7.2.Save model
8.test 函数
    8.1.记录 precision
9.Matplotlib 绘图
10.load model
11.inference


一个深度学习模型一般包含以下几个文件：
datasets文件夹：存放需要训练和测试的数据集
dataset.py：加载数据集，将数据集转换为固定的格式，返回图像集和标签集
model.py：根据自己的需求搭建一个深度学习模型，具体搭建方法参考
config.py：将需要配置的参数均放在这个文件中，比如batchsize，transform，epochs，lr等超参数
train.py:加载数据集，训练
predict.py：加载训练好的模型，对图像进行预测
requirements.txt:一些需要的库，通过pip install -r requirements.txt可以进行安装
readme：记录一些log
log文件：存放训练好的模型
loss文件夹：存放训练记录的loss图像
"""

In [6]:
# 1.张量tensor
"""
张量涵盖了一般意义上的标量，向量，矩阵乃至更高维的数据，是进行深度学习的基础，通常与numpy搭配使用

注：如非特地说明，本文中张量的阶和维概念一致
"""
import torch
import numpy as np

# 创建张量（还可使用Tensor函数创建默认为浮点型的张量）
A0 = torch.tensor(1) # 0阶张量（标量）
A1 = torch.tensor([1,2,3]) # 1阶张量（向量）
A2 = torch.tensor([ # 2阶张量（矩阵）
    [1,2,3],
    [4,5,6],
    [7,8,9]
])
A3 = torch.tensor([ # 3阶张量
    [[1,2,3],
     [4,5,6],
     [7,8,9]],

     [[1.1, 2.2, 3.3],
      [4.4, 5.5, 6.6],
      [7.7, 8.8, 9.9]
    ]])
A3.type() # 数据类型，返回'torch.FloatTensor'
A3.shape # 维度信息，返回torch.Size([2, 3, 3])


# 特殊张量的创建
A4 = torch.rand(3,3) # 0-1间的3x3均匀分布数据
A5 = torch.randn(3,3) # 均值为0，方差为1的3x3正态分布数据
"""
还可使用numpy先创建列表，再转换为tensor，如下：
A4 = np.random.uniform(size=9).reshape(3,3)
A4 = torch.tensor(A4)
A4 = numpy.array(A4) # 再次转换为numpy

tensor比之numpy优势在于可以采用显卡加速运算
"""
A6 = torch.full([2,3],1) # 2阶（2x3）的值全为1的张量
A7 = torch.arange(0,10,2) # 0到9的，步长为2的1阶张量
A8 = torch.linspace(0,10,steps=4) # 1阶张量，将0-10平均切分为4份
A9 = torch.eye(3) # 3x3单位矩阵

# 张量基本运算
# 切片运算
A = torch.randn(4,4,4)
A[1,[2,3],:3] # 获取第一阶上序号1，第二阶上序号2/3，以及第三阶上序号0/1/2的2x3数据

# 排序
A = torch.tensor([[1,4,3], [4,7,6], [10,9,7]])
torch.sort(A,0,descending=False) # 按张量第一阶升序排序
torch.topk(A,k=2,dim=0,largest=True) # 保留张量第一阶上前两个最大值

# 算术运算
A = torch.randn(4,4,4)
A+3; A-3 # 元素+3/-3
A.pow(2) # 平方
A.pow(0.5) # 开方
torch.exp(A) # 以e为底的指数
torch.log(A) # 以e为底的对数
torch.sum(A) # 所有元素求和，若为torch.sum(A,0)则表示按第一阶对元素求和
torch.mean(A) # 所有元素求均值，若为torch.mean(A,0)则表示按第一阶对元素求均值（仅用于浮点型张量）
torch.max(A) # 所有元素求最大值，其余同上
torch.argmax(A,0) # 沿着张量第一阶返回最大值的索引
torch.min(A) # 所有元素求最小值，其余同上
torch.argmin(A,0) # 沿着张量第一阶返回最小值的索引


# 广播运算
"""
两个张量进行广播运算的基本条件
1.两张量阶数及对应维数相同
2.两张量shape从后往前看，对应阶要么不存在，要么维数是1
"""
A = torch.randn(4,4,4)
B = torch.randn(1,4)
A+B
A-B 

# 乘积运算
A = torch.full((2,2,3),4)
B = torch.full((2,2,3),3)
C = torch.full((2,3,2),2)
A*B # 阶数和对应长度相同的张量，对应位置元素相乘
A@C # 张量相乘，最后两阶应遵循矩阵乘法规则

# 张量变换：reshape、squeeze、flatten、cat、stack、transpose/permute
# reshape重构张量的阶数和对应长度，与numpy中reshape功能一致，转换顺序由高阶到低阶，且转换前后元素个数不变
A = torch.randn(2,3,4) # 三阶（2x3x4）
A.reshape(6,4) # 二阶（6x4）
A.reshape(2,12) # 二阶（2x12）
A.reshape(1,2,3,4) # 四阶（1x2x3x4）
A.reshape(-1,2,2,2) # 第一阶的长度会自动根据张量元素个数计算，结果为四阶（3x2x2x2)

# squeeze压缩一个张量可以移除所有长度为1的轴axis，而unsqueeze解压一个张量则会在对应阶数增加一个长度
A = torch.randn(1,12) # 二阶（1x12）
A.squeeze() # 一阶（12）
A.unsqueeze(dim=2) # 三阶（1x12x1）

# flatten可以抹平张量对应阶以后所有阶数，将它完全展平，属于特殊的reshape函数
A = torch.randn(2,3,4) # 三阶（2x3x4）
A.flatten(start_dim=0) # 一阶（24）：将1阶及以上展平
A.flatten(start_dim=1) # 二阶（2x12）：将2阶及以上展平

# cat可以按对应阶进行张量拼接
A = torch.randn(2,3)
B = torch.rand(2,3) 
torch.cat((A,B),dim=1) # 二阶（2x6）：按一阶进行拼接，则二阶长度增加

# stack则会增加一个新的阶，并在该阶上对两个张量进行堆叠
A = torch.randn(2,3)
B = torch.rand(2,3)
torch.stack((A,B),dim=1) # 三阶（2x3x2）：按一阶进行堆叠
"""
tensor([[ 0.2775, -0.3184,  1.3068],
        [-0.3685, -0.0498, -1.2364]]) 
 tensor([[0.6206, 0.1927, 0.8001],
        [0.9898, 0.4323, 0.9723]])
tensor([[[ 0.2775,  0.6206],
         [-0.3184,  0.1927],
         [ 1.3068,  0.8001]], r

        [[-0.3685,  0.9898],
         [-0.0498,  0.4323],
         [-1.2364,  0.9723]]])
"""

# transpose和permute可用于交换张量各阶，矩阵转置是其体现之一
A = torch.randn(2,3,4)
A.transpose(1,2) # 只能交换（输入）两个阶，交换后维度信息为（2，4，3）
A.permute(1,2,0) # 能且必须交换（输入）所有阶，交换后维度信息为（3，4，2）
# 类似的，numpy中transpose函数也可用于交换维度，如三维数组A，可用np.transpose(A,(1,2,0))交换维度

In [ ]:
# 2.Dataset、DataLoader封装和加载数据集
"""
知识点：
Dataset： 覆写Dataset类用于对数据集进行封装，对数据进行预处理，清洗数据，记录 sample 与 label 的对应关系等等；
DataLoader ：Dataloader类用于将 Dataset 封装成迭代器，将数据向量化，使之更适合加载进入神经网络。
"""

# 2.1.加载Pytorch的torchvision模块自带数据集，以FashionMNIST为例
import torch
import torchvision
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np

# 训练集
train_data = datasets.FashionMNIST(
    root="Data",  # 数据存储路径
    train=True, # 下载训练集(True)或测试集(False)
    download=True, # 是否从互联网下载Pytorch自带的数据（若数据已存在则不会重复下载）
    transform=transforms.ToTensor() # 特征标签转换
)

# 测试集
test_data = datasets.FashionMNIST(
    root="Data", 
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

# 单一样本数据预览
sample = next(iter(train_data)) # 将train_data转为迭代器，并取第一个数据
"""
其中iter函数可用于将列表、字典、字符串等转换为迭代器（可用循环整体调用，也可用next函数逐个调用）
如 A = iter([1,2,3,4])
1. for i in A: print(i) 输出为1 2 3 4
2. next(A); next(A) 输出为1 2
"""
image,label = sample # image为图片数据，label为标签
print(image.shape) # 输出图片数据格式：torch.Size([1, 28, 28])，其中颜色通道数为1，长宽各为28个数据位
plt.imshow(image.squeeze(), cmap='gray') # 输出图片
print('label:',label)

# 批量样本数据预览
train_loader = torch.utils.data.DataLoader(train_data, batch_size=10) # 取10个样本
batch = next(iter(train_loader))
images,labels = batch
print(images.shape) # 输出为torch.Size([10, 1, 28, 28])，其中样本数为10
grid = torchvision.utils.make_grid(images,nrow=10) # 设置一个布局，将images中的图像按一行10个拼接输出
plt.figure(figsize=(15,15)) # 设定画布大小
plt.imshow(np.transpose(grid,(1,2,0))) # 调换图像各阶数据，将通道信息放在最后，便于显示
print(labels)

#--------------------------------------------------------------------------------------------#

# 2.2.加载本地数据集，以图像数据hymenoptera_data为例
"""
从上面加载官方数据集不难看出，加载本地数据集我们同样需得到以下结果：
【1】得到所有样本数据，样本数据标签及地址  
【2】得到数据集长度 
【3】能够得到指定位置或数量的数据集，以便后续的预处理操作 

蚂蚁蜜蜂数据集下载链接: https://pan.baidu.com/s/1jZoTmoFzaTLWh4lKBHVbEA 密码: 5suq
"""
import os
import numpy as np
from PIL import Image
from torchvision import transforms,utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

class MyData(Dataset):

    def __init__(self, root_dir, label_dir): # 初始化函数。提供数据地址和路径信息
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)
        self.img_path = os.listdir(self.path)
    
    def __len__(self): # 总样本数量
        return len(self.img_path)

    def __getitem__(self, index): # 取出指定位置的数据内容和标签信息
        img_item_name = self.img_path[index]
        img_item_path = os.path.join(self.root_dir,  self.label_dir, img_item_name)
        img = Image.open(img_item_path).convert('RGB')
        transform = transforms.Compose([  # 图像变换
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.2,0.2,0.2], std=[0.5,0.5,0.5])
                ])
        img = transform(img)
        return img, img_item_name

root_dir = 'C:\\Users\\XGQ\\Desktop\\Programs\\Python\\Python程序\\Learning\\Deep Learning\\Data\\hymenoptera_data\\train'
ants_label_dir = 'ants'
ants_dataset = MyData(root_dir, ants_label_dir)

# 单一样本数据预览
image, label = ants_dataset.__getitem__(0)
print('label:',label)
image = np.transpose(image,(1,2,0))
plt.imshow(image)

# 批量样本数据预览
loader = DataLoader(ants_dataset, batch_size=9, shuffle=True , num_workers=0, drop_last=False) # 将全部图片随机加载为9个一组的图片数据（不采取多线程加载，不丢弃剩余数据）
batch  = next(iter(loader)) # 取第一组图片数据
imgs,labels = batch # 取图片及标签

grid = utils.make_grid(imgs,nrow=3) # 设置一个布局，将images中的图像按一行3个拼接输出
plt.figure(figsize=(10,10)) # 设置画布大小
plt.imshow(np.transpose(grid,(1,2,0))) # 调换图像各阶数据，将通道信息放在最后，便于显示
for i in range(9): # 输出标签，3个为一行
    if (i+1)%3!=0:
        print(labels[i],end="\t")
    else:
        print(labels[i],end="\n")

#--------------------------------------------------------------------------------------------#

# 2.3.加载torchtext自带数据集，以文本数据为例

In [ ]:
# 3.PIL与OpenCV
"""
PIL是Python的内置模块，对于一般的图像处理来说已经够用，更专业的则往往要用到OpenCV
"""
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

# 3.1 PIL
# 读入、展示并保存猫猫图片
from PIL import Image
img = Image.open('path')
img.show()
img.save('path',format=None) # 其中path为保存路径，format（可选）为代转换的图片格式

# 输出类型、格式、尺寸、模式信息
print(type(img), "\n",img,format, "\n",img.size, "\n", img.mode)

# 图片通道分离
r,g,b = img.split() # 以三通道的jpg图像为例，可分割出R,G,B三通道的灰度图像

# 获取图片像素数据矩阵
matrix = np.array(img.getdata(band=None)) # 返回全部通道像素矩阵，其中band=0/1/2分别表示r,g,b单通道


# 3.2 OpenCV


In [ ]:
# 4.图像处理transform
"""
transform是深度学习torchvision模块下用于图像处理的专用模块，本文介绍它常用的几个函数并作演示
"""
from PIL import Image
from torchvision import transforms

# 打开图片
image = Image.open('C:\\Users\\XGQ\\Desktop\\Programs\\Python\\Python程序\\Learning\\Deep Learning\\Data\\pokeman\\pikachu\\00000000.jpg')
image.show()

# 提取出图片各通道的像素数据，并转换为张量格式
toTensor = transforms.ToTensor()
tensor_img = toTensor(image)
print(tensor_img)

# 将张量数据转为图片
toPILImage = transforms.ToPILImage()
img = toPILImage(tensor_img)
img.show()

# 调整图片大小（原理为图片的上下采样）
resize = transforms.Resize((600,500)) # 调整后高=600, 宽=500
resize_img = resize(img)
resize_img.show()

# 图片标准化
normalize = transforms.Normalize(mean=[0.2,0.2,0.2], std=[0.5,0.5,0.5])
norm_img = toPILImage(normalize(tensor_img))
norm_img.show()

# Compose整合各对象
image = Image.open('C:\\Users\\XGQ\\Desktop\\Programs\\Python\\Python程序\\Learning\\Deep Learning\\Data\\pokeman\\pikachu\\00000000.jpg')
transform = transforms.Compose([
    transforms.resize((600,500)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.2,0.2,0.2], std=[0.5,0.5,0.5])
    ])
img = transform(image)

In [ ]:
# 5.可视化工具tensorboard.SummaryWriter
"""
tensorboard原是TensorFlow的可视化工具，因方便好用故被Pytorch学习，从1.2.0版本开始支持tensorboard，前版本可用tensorboardX代替

tensorboard的工作原理：把深度学习项目中所关心的数据（包括图片，文字，图表等等）存入到某文件夹中，再取出该文件夹的数据展示于浏览器
需注意数据只能为tensor、np.array或string类型

展示方法：vscode有内置的tensorboard插件，也可在对应环境的终端输入tensorboard --logdir="文件夹名"
"""
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter 
from torchvision import datasets,transforms

# 下载并加载CTFAR10测试数据集
dataset = datasets.CIFAR10(root='../Deep Learning/Data', train=False, download=True, transform=transforms.ToTensor()) 
dataloader = DataLoader(dataset, batch_size=64)

# 设定SummaryWriter文件夹目录
writer = SummaryWriter('../Deep Learning/logs/log1')

# add_images函数
step = 0
for data in dataloader:
    imgs, labels = data
    """
    tag: 图像数据名
    img_tensor: 图像数据
    global_step: 全局步长值
    dataformats: 匹配图像数据格式（CHW为通道数+高+宽（默认）、HWC为高+宽+通道数、HW为高+宽）
    """
    writer.add_images(tag="images", img_tensor=imgs, global_step=step) # dataformats默认为size数+CHW
    step += 1
writer.close()

# add_scalar函数
for step in range(20):
    writer.add_scalar(tag="graph",scalar_value=step*3, global_step=step) # 单曲线绘图，scalar_value相当于图像Y值，global_step相当于X值
    writer.add_scalars(tag="graphs",tag_scalar_dict={'graph1':step*2, 'graph2':step*0.5}, global_step=step) # 多曲线绘图，tag_scalar_dict相当于图像Y值
writer.close()

In [ ]:
# 6.模型的查看，保存与加载
"""
深度学习模型的保存，通常是指模型训练过程中或训练结束后，将学习好的模型参数（不建议保存模型本身）保存起来留作调用。
深度学习模型的加载，是指在模型训练开始前，或者使用模型做预测时，将保存的模型参数加载到网络中
深度学习模型查看主要指网络结构和参数的查看
"""
model = "cnn"
optimizer =  torch.optim.SGD(model.parameters())
epoch = 100
loss = None

# 保存加载方式1(model.state_dict字典)
torch.save(model.state_dict(), '保存路径/model_state_dict.pth') # 网络参数保存
model.load_state_dict(torch.load('保存路径/model_state_dict.pth')) # 网络参数加载

# 保存加载方式2(checkpoint字典)
# 保存checkpoint（方式1的plus版本，还可保存优化器参数，损失等信息）
torch.save({
            'epoch': epoch,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
            'loss':loss}, 
            '保存路径/model_checkpoint.tar')   # 这里的后缀名官方推荐使用.tar            

# 加载checkpoint
checkpoint = torch.load('保存路径/model_checkpoint.tar')    # 加载checkpoint
model.load_state_dict(checkpoint['model_state_dict']) # 网络参数加载
optimizer.load_state_dict(checkpoint['optimizer_state_dict']) # 优化器参数加载
epoch = checkpoint['epoch'] # epoch加载
loss = checkpoint['loss'] # loss加载

# 模型的查看
for name, net in model.named_modules(): # 查看网络结构
    print(name,":",net)
for name, param in model.named_parameters(): # 查看网络参数，name为参数名
    print(name,":",param.shape) 
# 除此以外，还可对保存的参数如model.state_dict和optimizer.state_dict等进行查看

In [ ]:
# 7.模型参数的初始化

In [4]:
# 8.利用GPU来加速计算（没太搞懂）

tensor([[  71.3695,  272.2614, -188.8705,  ...,   61.2419, -212.0480,
         -108.5798],
        [ -21.6821,  192.0014,  107.6483,  ...,  141.7072,   84.5083,
           65.1910],
        [-186.6979,  -25.2006,   29.9123,  ..., -282.3174,  -70.1695,
           40.0141],
        ...,
        [   6.0081,   92.3619, -286.1390,  ...,  152.2972,   -8.6563,
          -44.9335],
        [ 143.2192,   42.6328,  -52.0900,  ...,   16.5236,  -79.0288,
           58.1762],
        [ -49.7891,   85.4641,  -35.1401,  ...,  -64.5795,  -20.2004,
           16.2153]])